In [1]:
#import pytest
import drjit as dr
import mitsuba as mi
import numpy as np
import csv
from sklearn.metrics import r2_score
from npy_append_array import NpyAppendArray
#mi.set_variant("cuda_ad_spectral")
mi.set_variant("llvm_ad_spectral")

[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [2]:
wls_n, refr_index_n, Cab_k_n, Car_k_n, Ant_k_n, Cbrown_k_n, Cw_k_n, Cm_k_n  = np.loadtxt(r'../../prospect-drjit/prospect_d_spectra.txt',unpack=True)
wls = mi.Float(wls_n)
refr_index =mi.Float(refr_index_n)
Cab_k=mi.Float(Cab_k_n)
Car_k=mi.Float(Car_k_n)
Ant_k=mi.Float(Ant_k_n)
Cbrown_k=mi.Float(Cbrown_k_n)
Cw_k=mi.Float(Cw_k_n)
Cm_k=mi.Float(Cm_k_n)

# Prospectd

In [3]:
def prospectd_wl(wl, Nleaf, Cab, Car, Cbrown, Cw, Cm, Ant):
    wl_index = wl-400
    Cab_abs = dr.gather(mi.Float,Cab_k,wl_index)
    Car_abs = dr.gather(mi.Float,Car_k,wl_index)
    Cbrown_abs = dr.gather(mi.Float,Cbrown_k,wl_index)
    Cw_abs = dr.gather(mi.Float,Cw_k,wl_index)
    Cm_abs = dr.gather(mi.Float,Cm_k,wl_index)
    Ant_abs = dr.gather(mi.Float,Ant_k,wl_index)
    n = dr.gather(mi.Float,refr_index,wl_index)

    k = (Cab * Cab_abs + Car * Car_abs + Cbrown * Cbrown_abs
         + Cw * Cw_abs + Cm * Cm_abs + Ant * Ant_abs) / Nleaf
    # fast cal
    trans_case1 = 1
    
    xx = 0.5 * k - 1.0
    yy = (((((((((((((((-3.60311230482612224e-13
        * xx + 3.46348526554087424e-12) * xx-2.99627399604128973e-11)
        * xx + 2.57747807106988589e-10) * xx-2.09330568435488303e-9)
        * xx + 1.59501329936987818e-8) * xx-1.13717900285428895e-7)
        * xx + 7.55292885309152956e-7) * xx-4.64980751480619431e-6)
        * xx + 2.63830365675408129e-5) * xx-1.37089870978830576e-4)
        * xx + 6.47686503728103400e-4) * xx-2.76060141343627983e-3)
        * xx + 1.05306034687449505e-2) * xx-3.57191348753631956e-2)
        * xx + 1.07774527938978692e-1) * xx-2.96997075145080963e-1
    yy = (yy * xx + 8.64664716763387311e-1) * xx + 7.42047691268006429e-1
    yy = yy - dr.log(k)
    trans_case2 = (1.0 - k) * dr.exp(-k) + k**2 * yy
    
    xx = 14.5 / (k + 3.25) - 1.0
    yy = (((((((((((((((-1.62806570868460749e-12
        * xx - 8.95400579318284288e-13) * xx - 4.08352702838151578e-12)
        * xx - 1.45132988248537498e-11) * xx - 8.35086918940757852e-11)
        * xx - 2.13638678953766289e-10) * xx - 1.10302431467069770e-9)
        * xx - 3.67128915633455484e-9) * xx - 1.66980544304104726e-8)
        * xx - 6.11774386401295125e-8) * xx - 2.70306163610271497e-7)
        * xx - 1.05565006992891261e-6) * xx - 4.72090467203711484e-6)
        * xx - 1.95076375089955937e-5) * xx - 9.16450482931221453e-5)
        * xx - 4.05892130452128677e-4) * xx - 2.14213055000334718e-3
    yy = ((yy * xx - 1.06374875116569657e-2)
             * xx - 8.50699154984571871e-2) * xx + 9.23755307807784058e-1
    yy = dr.exp(-k) * yy / k
    trans_case3 = (1.0 - k) * dr.exp(-k) + k**2 * yy
    
    trans = dr.select(k<=0.0, trans_case1, dr.select(k<=4.0, trans_case2, dr.select(k<=85.0,trans_case3, 0)))
    
    
    # reflectance and transmittance of one layer
    talf = tav_wl(40., n)
    ralf = 1.0 - talf
    t12 = tav_wl(90., n)
    r12 = 1. - t12
    t21 = t12 / n ** 2
    r21 = 1 - t21

    # top surface side
    denom = 1. - r21 ** 2 * trans ** 2
    Ta = talf * trans * t21 / denom
    Ra = ralf + r21 * trans * Ta

    # bottom surface side
    t = t12 * trans * t21 / denom
    r = r12 + r21 * trans * t
    
    # reflectance and transmittance of multiple layers
    D = dr.sqrt((1. + r + t) * (1. + r - t) * (1. - r + t) * (1. - r - t))
    a = (1. + r ** 2 - t ** 2 + D) / (2. * r)
    b = (1. - r ** 2 + t ** 2 + D) / (2. * t)

    bNm1 = dr.power(b, Nleaf - 1.)
    bN2 = bNm1 ** 2
    a2 = a ** 2
    denom = a2 * bN2 - 1.
    Rsub = a * (bN2 - 1.) / denom
    Tsub = bNm1 * (a2 - 1.) / denom
    # Case of zero absorption
    Tsub = dr.select(r + t >= 1.,t / (t + (1. - t) * (Nleaf - 1.)), Tsub)
    Rsub = dr.select(r + t >= 1.,1. - Tsub, Rsub)
    # Reflectance and transmittance of the leaf: combine top layer with next N-1 layers
    denom = 1. - Rsub * r
    tran = Ta * Tsub / denom
    refl = Ra + Ta * Rsub * t / denom

    return refl, tran

def tav_wl(theta, ref):
    rd = dr.pi/180
    r2 = ref ** 2.0
    rp = r2 + 1.0
    rm = r2 - 1.0
    a = ((ref + 1.0) ** 2.0) / 2.0
    k = -(r2 - 1.0) ** 2.0 / 4.0
    sa = dr.sin(theta*rd)
    b1 = dr.select(theta == 90.0, 0, dr.sqrt((sa ** 2.0 - rp / 2.0) ** 2.0 + k))
                     
    k2 = k ** 2.0
    rm2 = rm ** 2.0
    b2 = sa ** 2.0 - rp / 2.0
    b = b1 - b2
                     
    ts = (k2 / (6.0 * b ** 3.0) + k / b - b / 2.0) - (k2 / (6.0 * a ** 3.0) + k / a - a / 2.0)
    tp1 = -2.0 * r2 * (b - a) / (rp ** 2.0)
    tp2 = -2.0 * r2 * rp * dr.log(b / a) / rm2
    tp3 = r2 * (b ** -1.0 - a ** -1.0) / 2.0
    tp4 = 16.0 * r2 ** 2.0 * (r2 ** 2.0 + 1.0) * dr.log((2.0 * rp * b - rm2) / (2.0 * rp * a - rm2)) / (rp ** 3.0 * rm2)
    tp5 = 16.0 * r2 ** 3.0 * ((2.0 * rp * b - rm2) ** -1.0 - (2.0 * rp * a - rm2) ** -1.0) / rp ** 3.0
    tp = tp1 + tp2 + tp3 + tp4 + tp5
    f = (ts + tp) / (2.0 * sa ** 2.0)

    return f


In [4]:
Nleaf = mi.Float(1.534077645)
Cab = mi.Float(44.710243902439)
Car = mi.Float(11.6409756097561)
Cbrown = mi.Float(0)
Cw = mi.Float(0.00985365853658537)
Cm = mi.Float(0.0037)
Ant = mi.Float(0)
wavelength = mi.Int([490,560,665,865])

key1 = 'leaf1.reflectance.values'
key2 = 'leaf1.transmittance.values'

def mse(image,image_ref):
    return dr.mean(dr.sqr(image - image_ref))*1e10

def mse_deng(image1,image2):
    return dr.mean((image1-image_ref)*(image2-image_ref))

def applyChange(bands,params,opt):
    ref,trans = prospectd_wl(bands, opt['Nleaf'], opt['Cab'], opt['Car'], Cbrown, opt['Cw'], opt['Cm'], Ant)
    key1 = 'leaf1.reflectance.values'
    key2 = 'leaf1.transmittance.values'
    params[key1]=ref
    params[key2]=trans
    params.update()

def image_ref_genertate(band,filepath):
    scene = mi.load_file(filepath)
    params = mi.traverse(scene)
    ref,trans = prospectd_wl(band, Nleaf, Cab, Car, Cbrown, Cw, Cm, Ant)
    
    params[key1] = ref
    params[key2] = trans
    params.update()
    image_ref = mi.render(scene,params)
    return params,image_ref,scene

def iterate(bands,filepaths):
    
    dr.enable_grad(Nleaf)
    dr.enable_grad(Cab)
    
    
    params,image_ref,scene = image_ref_genertate(bands,filepaths)
    
    print('ref:',np.array(Nleaf),np.array(Cab),np.array(Car),np.array(Cw),np.array(Cm),np.array(params[key1]),np.array(params[key2]))

    opt = mi.ad.Adam(lr=0.05)
    opt['Nleaf'] = mi.Float(1.5)
    opt['Cab'] = mi.Float(50.0)
    opt['Car'] = mi.Float(10.0)
    opt['Cw'] = mi.Float(0.01)
    opt['Cm'] = mi.Float(0.01)
    opt.set_learning_rate({'Nleaf':0.03,'Cab':2.0,'Car':0.1,'Cw':0.005,'Cm':0.001})
    #applyChange(bands,params,opt)
    #image_init = mi.render(scene, params)
    
    iteration_count = 300
    errors = []
    nleaferrors = []
    caberrors = []
    carerrors = []
    cmerrors = []
    cwerrors = []
    lossarr=[]
    for it in range(iteration_count):
        # Perform a (noisy) differentiable rendering of the scene
            opt['Nleaf'] = dr.clamp(opt['Nleaf'], 1.0, 3.0)
            opt['Cab'] = dr.clamp(opt['Cab'], 0.0001, 100.0)
            opt['Car'] = dr.clamp(opt['Car'], 0.0001, 30.0)
            opt['Cw'] = dr.clamp(opt['Cw'], 0.00005, 0.04)
            opt['Cm'] = dr.clamp(opt['Cm'], 0.002, 0.018)
            applyChange(bands,params,opt)
            image = mi.render(scene, params)
            loss = mse(image,image_ref)

            # Backpropagate through the rendering process
            dr.backward(loss)

            # Optimizer: take a gradient descent step
            opt.step()

            # Track the difference between the current color and the true value
            err_ref = dr.sum(dr.abs(opt['Nleaf'] - Nleaf))
            err_ref += dr.sum(dr.abs(opt['Cab'] - Cab))
            err_ref += dr.sum(dr.abs(opt['Car'] - Car))
            err_ref += dr.sum(dr.abs(opt['Cw'] - Cw))
            err_ref += dr.sum(dr.abs(opt['Cm'] - Cm))
            print(f"Iteration {it:02d}: loss error= {loss}, parameter error = {err_ref[0]:6f}, Nleaf={opt['Nleaf']},Cab={opt['Cab']},Car={opt['Car']},,Cw={opt['Cw']},,Cw={opt['Cm']}, key1={params[key1]}, key2={params[key2]}", end="\r")
    print('\nOptimization complete.')
    
  
    print('ref:',Nleaf,Cab,Car,Cw,Cm)
    print('inverse:',opt['Nleaf'],opt['Cab'],opt['Car'],opt['Cw'],opt['Cm'])
    src = [Nleaf,Cab,Car,Cw,Cm]
    pred=[opt['Nleaf'],opt['Cab'],opt['Car'],opt['Cw'],opt['Cm']]
    return np.array(src).flatten(), np.array(pred).flatten()

In [ ]:
bands=mi.Int([490,560,665,865])
data= np.load('./lopexdataset.npy')
specis= data.tolist()
y_true = []
y_pred = []
for spec in specis[0:20]:  # first 20 groups.
    Nleaf = mi.Float(spec[0])
    Cab = mi.Float(spec[1])
    Car = mi.Float(spec[2])
    Cbrown = mi.Float(0)
    Cw = mi.Float(spec[3])
    Cm = mi.Float(spec[4])
    Ant = mi.Float(0)
    src,pred=iterate(bands,'../Parameters/inverse_allband.xml')
    with NpyAppendArray('src.npy') as npaa:
        npaa.append(src)
    with NpyAppendArray('pred.npy') as npaa1:
        npaa1.append(pred)
    y_true.append(src.tolist())
    y_pred.append(pred.tolist())
print(y_true)
print(y_pred)
